### Part 2:

I completely misread part 2 and for a few days have misunderstood it.

- My initial read was that a basin was only composed of neighbors flowing downwards (meaning I needed to search only for neighbors with an increasing value). 
    - However, it looks like the definition is looser and is just going to be all points that are not a 9. 
    - I am either still misunderstanding or it looks like the example is a bit misleading (it seems like it follows a pattern of each added point being an increase in value up to 9?).
    - Regardless, my original recursive function was handling the test input but failing on main output. I eventually reread `"Locations of height 9 do not count as being in any basin, and all other locations will always be part of exactly one basin."` Assumed in here is that input data will segment properly so that 9s and general boundary can be used to ensure no duplicates across basins so I didn't even bother checking for that. 

- Another error I had: 
    - I was looking at diagonals, but it clearly states in part 1 that diagonals should not be considered. 

#### Recursive function:

Finally cracked a recursion function in AoC!
- I take a single point (assume its lowpoint passed in)
- Then look at the 4 possible neighbors
- if the neighbors are within the border and not already accounted for then I proceed to check if 9.
- if not 9, then pass coordinates into function and do the same search as above
    - once it runs out of points to search it outputs a set
    - this set updates the other set, and that allows us to recursively grow a set of pointss

In [1]:
import numpy as np 
import time 


def searchAreaPt(i,j, m):
    """Receive input i,j and search 4 points around.
    Return 1 to mark our basin, which is a point where all neighbors are greater
    """
    # start
    v = m[i,j]
    
    # search around    
    elements = [(i, j-1), (i-1,j),(i+1,j), (i, j+1)]
    count = 0
    for e in elements:
        if (e[0] < 0) or (e[1] < 0): # step needed to avoid wrapping functionality
            continue 
        else:
            try:
                if v >= m[e]:
                    count += 1
            except:
                continue

    if count > 0:
        return 0
    return 1

def recursiveSearchL(i,j, m, full_list = None):
    """
    Recursively checks for elements in all 4 directions to see if next element exists / is not a 9
    
    The recursive aspect: if we find a point touching our current point then we want to explore that point &
    it's respective neighbors. This will continue until we either run into only 9s or exceed boundaries
    
    """
    # adding start to set (tried avoiding but led to recursion depth issues - which i don't understand
    full_list.add((i,j))
    
    # search area
    elements = [(i, j-1), (i-1,j),(i+1,j), (i, j+1)]
    
    # check if v has any neighbors that are a 9 or exceed boundaries, in which case we ignore
    for i,j in elements:
        if (i < 0) or (j < 0) or (i >= m.shape[0]) or (j >= m.shape[1]) or ((i,j) in full_list):
            continue
        else:
            if ((m[i,j] != 9)):
                full_list.update(recursiveSearchL(i,j,m, full_list))
            else:
                continue
    return full_list

In [2]:

with open('data/day09_test.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]

# get matrix sizing
row = len(data)
col = len(data[0])
print(f"Building {row} x {col} matrix")

# store as matrix of size row x col
m = np.asarray([int(x) for x in ''.join(data)])
m.shape = (row,col)

# Running search 
tot_sum = 0
lowpoint = []
for r in range(row):
    for c in range(col):
        if searchAreaPt(r,c,m):
            lowpoint.append((r,c))

print(f"Low points found: {lowpoint}")

# print example
size_list = []
for i,j in lowpoint:
    print(f"Lowpoint: ({i}, {j})")
    full_list = recursiveSearchL(i = i,j = j, m = m, full_list = set())
    len_list = len(full_list)

    print(set(full_list))
    size_list.append(len_list)

size_list.sort()
print(size_list)
f = 1
for z in size_list[-3:]:
    f *= z
print(f)

Building 5 x 10 matrix
Low points found: [(0, 1), (0, 9), (2, 2), (4, 6)]
Lowpoint: (0, 1)
{(0, 1), (1, 0), (0, 0)}
Lowpoint: (0, 9)
{(0, 7), (1, 8), (0, 9), (0, 6), (2, 9), (0, 5), (1, 6), (0, 8), (1, 9)}
Lowpoint: (2, 2)
{(1, 3), (2, 4), (1, 2), (2, 1), (3, 4), (3, 1), (1, 4), (3, 0), (2, 3), (3, 3), (2, 2), (3, 2), (2, 5), (4, 1)}
Lowpoint: (4, 6)
{(3, 8), (2, 7), (4, 9), (3, 7), (4, 6), (4, 5), (4, 8), (3, 6), (4, 7)}
[3, 9, 9, 14]
1134


### Part 2: Actual

In [3]:
# read data 
with open('data/day09.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]

# get matrix sizing
row = len(data)
col = len(data[0])
print(f"Building {row} x {col} matrix")

# store as matrix of size row x col
m = np.asarray([int(x) for x in ''.join(data)])
m.shape = (row,col)

# Running search for lowpoints
tot_sum = 0
lowpoint = []
for r in range(row):
    for c in range(col):
        if searchAreaPt(r,c,m):
            lowpoint.append((r,c))

print(f"Total Low points found: {len(lowpoint)}")

# For each lowpoint determining size of basin
size_list = []
for i,j in lowpoint:
    full_list = recursiveSearchL(i = i,j = j, m = m, full_list = set())
    len_list = len(full_list)

    size_list.append(len_list)
size_list.sort()

# output
f = 1
for z in size_list[-3:]:
    f *= z
print(f"3 largest basins: {size_list[-3:]}")
print(f)

Building 100 x 100 matrix
Total Low points found: 224
3 largest basins: [95, 99, 101]
949905
